In [2]:
!pip install -r requirements.txt

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


# Run code with setup

In [2]:
"GRU", "LSTM", "GCN", "TGCN","TGCN_LSTM","TGCN_UGRNN"

('GRU', 'LSTM', 'GCN', 'TGCN', 'TGCN_LSTM', 'TGCN_UGRNN')

In [2]:
["mse_with_regularizer","mse_with_regularizer_l1"]

['mse_with_regularizer', 'mse_with_regularizer_l1']

In [ ]:
"losloop","PEMS08","PEMS04",""

In [ ]:
from subprocess import Popen,PIPE
model = ["TGCN"]
max_epochs = "1"
learning_rate = "0.001" 
weight_decay ="0"
enable_progress_bar = "False"
batch_size = "64"
hidden_dim = "64"
cell_dim = "64"
dropout = "0"
layer_2 = "1"
loss = ["mse"]
data = ["losloop"]
gpus = "0"
for i in data:
    for j in loss:
        for k in model:
            for l in layer_2:
                print("Dataset:",i," Loss:",j," Model:",k,"Layer 2:",l)
                cmd_str = ["python",
                       "main.py",
                       "--enable_progress_bar",enable_progress_bar,
                       "--model_name",k,
                       "--max_epochs",max_epochs,
                       "--learning_rate",learning_rate,
                       "--weight_decay",weight_decay,
                       "--batch_size",batch_size,
                       "--hidden_dim",hidden_dim,
                       "--cell_dim",cell_dim,
                       "--dropout",dropout,
                       "--loss",j,
                       "--layer_2",l,
                       "--data",i,
                       "--gpus",gpus,
                      ]
                p = Popen(cmd_str,stdout=PIPE)
                for m in p.stdout:
                    print(m)

Dataset: losloop  Loss: mse_with_regularizer_entropy  Model: TGCN Layer 2: False
b'--------------------------------------------------------------------------------\n'
b'DATALOADER:0 VALIDATE RESULTS\n'
b"{'ExplainedVar': 0.2797713279724121,\n"
b" 'MAE': 12.394514083862305,\n"
b" 'R2': 0.16201472282409668,\n"
b" 'RMSE': 16.684919357299805,\n"
b" 'accuracy': 0.7159579396247864,\n"
b" 'val_loss': 448240805740544.0}\n"
b'--------------------------------------------------------------------------------\n'
Dataset: losloop  Loss: mse  Model: TGCN Layer 2: False


# Test one with time 

In [9]:
import time

In [7]:
%%time

!python main.py \
--model_name TGCN \
--data losloop \
--max_epochs 2 \
--learning_rate 0.001 \
--weight_decay 0 \
--batch_size 64 \
--hidden_dim 64 \
--self_attention 1 \
--loss "mse" \
--seq_len 12 \
--pre_len 12 \
--svd 0

[2023-05-06 05:57:44,318 INFO]{'logger': True, 'checkpoint_callback': None, 'enable_checkpointing': True, 'default_root_dir': None, 'gradient_clip_val': None, 'gradient_clip_algorithm': None, 'process_position': 0, 'num_nodes': 1, 'num_processes': 1, 'devices': None, 'gpus': None, 'auto_select_gpus': False, 'tpu_cores': None, 'ipus': None, 'log_gpu_memory': None, 'progress_bar_refresh_rate': None, 'enable_progress_bar': True, 'overfit_batches': 0.0, 'track_grad_norm': -1, 'check_val_every_n_epoch': 1, 'fast_dev_run': False, 'accumulate_grad_batches': None, 'max_epochs': 2, 'min_epochs': None, 'max_steps': -1, 'min_steps': None, 'max_time': None, 'limit_train_batches': 1.0, 'limit_val_batches': 1.0, 'limit_test_batches': 1.0, 'limit_predict_batches': 1.0, 'val_check_interval': 1.0, 'flush_logs_every_n_steps': None, 'log_every_n_steps': 50, 'accelerator': None, 'strategy': None, 'sync_batchnorm': False, 'precision': 32, 'enable_model_summary': True, 'weights_summary': 'top', 'weights_sav

# Results to save

In [ ]:
DATALOADER:0 VALIDATE RESULTS
{'ExplainedVar': 0.6397898197174072,
 'MAE': 5.414433479309082,
 'R2': 0.6379294991493225,
 'RMSE': 8.367987632751465,
 'accuracy': 0.8574607968330383,
 'val_loss': 70.0232162475586}
--------------------------------------------------------------------------------
Validating: 100%|█████████████████████████████████| 1/1 [00:00<00:00,  1.25it/s]

CPU times: user 14.8 s, sys: 3.32 s, total: 18.1 s
Wall time: 17min 59s


# SVD 

In [ ]:
import pandas as pd
import numpy as np
import torch 
import time

def svd_low_rank_approximation(matrix,k):
    # Perform SVD on the matrix
    u, s, v = torch.linalg.svd(matrix, full_matrices=False)
    
    k=0
    # Truncate the singular values and matrices to rank k
    s_k = s[:k]
    u_k = u[:, :k]
    v_k = v[:, :k]
    return u_k.mm(torch.diag(s_k)),v_k.t()

def adjmat(adj):
    return torch.mm(torch.mm(adj[1], torch.diag(adj[0])), adj[2].t())

PEMS08_adj = pd.read_csv(r'data/pems08_adj.csv',header=None)
adj = np.array(PEMS08_adj,dtype=np.float32)
adj = torch.tensor(adj)

data = np.load("data/pems08.npz")
data = data.f.data
df_data = pd.DataFrame(data[:,:,0])
df_data = df_data.values.tolist()
feat = np.array(df_data, dtype=np.float32)
feat = torch.tensor(feat)

for i in range(1):
    adjsvd= svd_low_rank_approximation(feat,115)
    adjmatrix1= adjsvd
for i in range(1):
    adjsvd= svd_low_rank_approximation(feat,170)
    adjmatrix= adjsvd
q=adjmatrix1[1].matmul(adj)
l=adjmatrix1[0].matmul(q)
start = time.time()

print("svd s u v = ",adjmatrix1[0].matmul(adjmatrix1[1].matmul( adj))
end = time.time()
print(end-start)
start = time.time()
print("svd s u v = ",l==adjmatrix1[0].matmul(adjmatrix1[1].matmul( adj)))
end = time.time()
print(end-start)

In [ ]:
import numpy as np
from numpy import linalg as la
import pandas as pd
np.random.seed(42)


def flip_signs(A, B):
    """
    utility function for resolving the sign ambiguity in SVD
    http://stats.stackexchange.com/q/34396/115202
    """
    signs = np.sign(A) * np.sign(B)
    return A, B * signs


# Let the data matrix X be of n x p size,
# where n is the number of samples and p is the number of variables
n, p = 170, 170
adj_df = pd.read_csv("data/pems08_adj.csv", header=None)
X = np.array(adj_df, dtype=np.float32)
X -= np.mean(X,axis=0)

# the p x p covariance matrix
C = np.cov(X, rowvar=False)
print("C = \n", C)
# C is a symmetric matrix and so it can be diagonalized:
l, principal_axes = la.eig(C)
# sort results wrt. eigenvalues
idx = l.argsort()[::-1]
l, principal_axes = l[idx], principal_axes[:, idx]
# the eigenvalues in decreasing order
print("l = \n", l)
# a matrix of eigenvectors (each column is an eigenvector)
print("V = \n", principal_axes)
# projections of X on the principal axes are called principal components
principal_components = X.dot(principal_axes)
print("Y = \n", principal_components)

# we now perform singular value decomposition of X
# "economy size" (or "thin") SVD
U, s, Vt = la.svd(X, full_matrices=False)
V = Vt.T
S = np.diag(s)

# 1) then columns of V are principal directions/axes.
#assert np.allclose(V, principal_axes)

# 2) columns of US are principal components
assert np.allclose(*flip_signs(U.dot(S), principal_components))

# 3) singular values are related to the eigenvalues of covariance matrix
assert np.allclose((s ** 2) / (n - 1), l)

# 8) dimensionality reduction
k = 2
PC_k = principal_components[:, 0:k]
US_k = U[:, 0:k].dot(S[0:k, 0:k])
assert np.allclose(*flip_signs(PC_k, US_k))

# 10) we used "economy size" (or "thin") SVD
assert U.shape == (n, p)
assert S.shape == (p, p)
assert V.shape == (p, p)
